# Instalar dependencias

In [1]:
%pip install sklearn

# Instalar Theano

In [2]:
%pip install --upgrade --no-deps theano

# Instalar Tensorflow y Keras


In [3]:
%pip install keras
%pip install tensorflow

# Redes Neuronales Artificales

# Cómo importar las librerías


In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Importar el data set


In [5]:
dataset = pd.read_csv('./Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

# Parte 1 - Pre procesado de datos

# Codificar datos categóricos

In [6]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

onehotencoder = ColumnTransformer(
    [('one_hot_encoder', OneHotEncoder(categories='auto'), [1])],   
    remainder='passthrough'                        
)
X = onehotencoder.fit_transform(X)
# Ojo a la trampa de las variables ficticias: https://statologos.com/trampa-de-variables-ficticias/#:~:text=Trampa%20de%20variable%20ficticia%20%3A%20cuando,regresi%C3%B3n%20y%20los%20valores%20p.
# La eliminamos quitando la primera columna
X = X[:, 1:]

# Con la columna de sexo, que ahora es la 3, no hace falta hacer variables dummy, podemos directamente hacer el LabelEncoder (conversión de un género a 0 y otro a 1)
# Si lo pensamos, al hacer el proceso de convertir a variables dummy hay que eliminar una variable ficticia por lo que en las variables que solo pueden tener dos 
# categorías no tiene sentido hacer el proceso de variables dummy
labelencoder_X_2 = LabelEncoder()
X[:, 3] = labelencoder_X_2.fit_transform(X[:, 3])

# Dividir el data set en conjunto de entrenamiento y conjunto de testing

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Escalado de variables

In [8]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

# Parte 2 - Construir la RNA

# Importar Keras y librerías adicionales


In [9]:
# import keras
from keras.models import Sequential
from keras.layers import Dense

# Inicializar la RNA

In [10]:
classifier = Sequential()

# Añadir las capas de entrada y primera capa oculta


In [11]:
# Dense representa la sinapsis, conexión entre capas
#   - Units es el número de nodos que queremos en la primera capa oculta. Un consejo para establecer el número de nodos en la capa oculta es hacer la media entre el 
#     número de nodos de entrada y de salida. Aquí tenemos (11 + 1) / 2 = 6
#   - kernel_initializer se utiliza para inicializar los pesos. Se utiliza una distribución uniforme. Serán pequeños y cercanos a 0.
#   - input_dim es el número de nodos de entrada que se tienen. input shape es un tensor, hay que especificarle el tamaño de la muestra y la dimensión de cada dato (en este caso 11)
classifier.add(Dense(units = 6, kernel_initializer = "uniform", activation = "relu", input_dim = 11))

# Añadir la segunda capa oculta

In [12]:
# En la segunda capa no hace falta input_dim ya que al venir de la anterior ya sabe que sería 6.
classifier.add(Dense(units = 6, kernel_initializer = "uniform",  activation = "relu"))

# Añadir la capa de salida

In [13]:
# - Al ser un resultado binario, se va o se queda del banco, solo habrá un nodo en la capa de salida.
#   Si se quisiese clasificar en tres categorías: cliente activo, pasivo, neutro en el banco...tendríamos 3 nodos. Ya no se usaría la sigmoide por no ser la más adecuada...
#   Deberíamos usar un escalón o un relu. Podríamos mantener la sigmoide pero probablemente tendríamos que usar softmax para que todas las probabilidades sumasen 1.
# - En la última capa utilizamos una función de activación sigmoide para tener la probabilidad de que la predicción pertenezca a una clase u otra.
classifier.add(Dense(units = 1, kernel_initializer = "uniform",  activation = "sigmoid"))

# Compilar la RNA

In [14]:
# Optimizer: es el algoritmo que se utiliza para encontrar el conjunto óptimo de pesos. Gradiente descendiente, gradiente descendiente estocástico, adam...
# loss (función de pérdidas): función que minimiza el error entre la predicción de la red y el valor real. ordinary least squares (minimización diferencias al cuadrado),
# Usamos la binary_crossentropy ya que es la más interesante para clasificación binaria.
# metrics: son las métricas que el sistema va a evaluar y las que va a intentar aumentar de una iteración a la siguiente.
classifier.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])

# Ajustamos la RNA al Conjunto de Entrenamiento

In [15]:
# batch_size: si recordamos la teoría, una vez que hemos hecho una pasada se actualiza el conjunto de pasos. Aquí definimos el tamaño del lote.
# epochs: definimos las veces que la red procesa todo el conjunto de datos para aprender.
classifier.fit(X_train, y_train,  batch_size = 10, epochs = 100)

Epoch 1/100
800/800 [==============================] - 1s 762us/step - loss: 0.4880 - accuracy: 0.7954
Epoch 2/100
800/800 [==============================] - 1s 743us/step - loss: 0.4286 - accuracy: 0.7960
Epoch 3/100
800/800 [==============================] - 1s 931us/step - loss: 0.4228 - accuracy: 0.8011
Epoch 4/100
800/800 [==============================] - 1s 755us/step - loss: 0.4166 - accuracy: 0.8257
Epoch 5/100
800/800 [==============================] - 1s 740us/step - loss: 0.4119 - accuracy: 0.8313
Epoch 6/100
800/800 [==============================] - 1s 775us/step - loss: 0.4085 - accuracy: 0.8307
Epoch 7/100
800/800 [==============================] - 1s 758us/step - loss: 0.4058 - accuracy: 0.8338
Epoch 8/100
800/800 [==============================] - 1s 859us/step - loss: 0.4039 - accuracy: 0.8360
Epoch 9/100
800/800 [==============================] - 1s 727us/step - loss: 0.4027 - accuracy: 0.8338
Epoch 10/100
800/800 [==============================] - 1s 751us/step - l

# Parte 3 - Evaluar el modelo y calcular predicciones finales

# Predicción de los resultados con el Conjunto de Testing

In [44]:
y_pred  = classifier.predict(X_test)
# Recordemos que las salidas son probabilidades, las convertimos a booleano
y_pred = (y_pred>0.5)

63/63 [==============================] - 0s 710us/step


# Elaborar una matriz de confusión

In [48]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(f"Precisión: {(cm[0][0] + cm[1][1]) / cm.sum()}")

[[1520   75]
 [ 200  205]]
Precisión: 0.8625


# Ejercicio

Utiliza nuestro modelo de RNA para predecir si el cliente con la siguiente información abandonará el banco:

*   Geografia: Francia
*   Puntaje de crédito: 600
*   Género masculino
*   Edad: 40 años de edad
*   Tenencia: 3 años.
*   Saldo: $ 60000

*   Número de productos: 2
*   ¿Este cliente tiene una tarjeta de crédito? Sí
*   ¿Es este cliente un miembro activo? Sí
*   Salario estimado: $ 50000

Entonces, ¿deberíamos decir adiós a ese cliente?

In [57]:
# 1. Creamos los datos
client_raw = pd.DataFrame([
  { 
    "CreditScore": 600, 
    "Geography": 'France', 
    "Gender": 'Male', 
    "Age": 40, 
    "Tenure": 3, 
    "Balance": 60000, 
    "NumOfProducts": 2, 
    "HasCrCard": 1, 
    "IsActiveMember": 1, 
    "EstimatedSalary": 50000 
  }
])
# 2. Transformamos los datos
client_tr = onehotencoder.transform(client_raw.to_numpy())[:, 1:]
client_tr[:,3] = labelencoder_X_2.transform(client_tr[:,3])

# 3. Escalamos los datos
client_tr = sc_X.transform(client_tr)

# 4. Predecimos
client_pred = classifier.predict(client_tr)
print(f"El cliente se queda con una probabilidad de {100 - client_pred[0][0] * 100}%")
client_pred = client_pred > 0.5

# 5. Mostramos resultado
result = "Si" if client_pred[0] else "No"
print(f"¿Deberíamos decir adiós a ese cliente? => {result}")


1/1 [==============================] - 0s 16ms/step
El cliente se queda con una probabilidad de 93.59163343906403%
¿Deberíamos decir adiós a ese cliente? => No
